# Spectrum Extraction

- author : Sylvie Dagoret-Campagne
- affiliation : LAL/IN2P3/CNRS
- date : March 21th 2017
- update :



In [74]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import matplotlib as mpl
import re
import pandas as pd

In [75]:
# to enlarge the sizes
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (10, 7),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)


## Constants for conversions

In [76]:
m_to_mm=1000.
mm_to_m=1e-3
inch_to_mm=25.4
mm_to_inch=1./inch_to_mm
micr_to_m=1e-6
m_to_micr=1./micr_to_m
m_to_cm=100.
m_to_nm=1e9
nm_to_m=1./m_to_nm

In [77]:
arcdeg_to_arcmin=60.
arcmin_to_arcdeg=1./arcdeg_to_arcmin
arcmin_to_arcsec=60.
arcdeg_to_arcsec=arcdeg_to_arcmin*arcmin_to_arcsec
arcsec_to_arcdeg=1./arcdeg_to_arcsec

In [78]:
deg_to_rad=np.pi/180.

In [79]:
rad_to_deg=1./deg_to_rad
rad_to_arcsec=rad_to_deg*arcdeg_to_arcsec
rad_to_arcmin=rad_to_deg*arcdeg_to_arcmin

## Configuration parameters at the telescope
--------------------------------------------------------

#### Telescope

In [80]:
Tel_Focal_Length=12.6 # m : Focal length of the telescope
Tel_Diameter=0.9 # m : Diameter of the telescope    

In [81]:
Tel_Fnum=Tel_Focal_Length/Tel_Diameter

#### Grating

In [82]:
Ronchi_r=200 # number of lines per mm
Ronchi_a=mm_to_m/Ronchi_r  # line spacing in meter unit
Ronchi_size=2*inch_to_mm*mm_to_m # size of Ronchi Grating in meter
Ronchi_N=Ronchi_size/Ronchi_a   # total number of lines (not used)

#### Filter

In [83]:
Filt_D=0.05769 # m distance of the filter position wrt CCD plane
Filt_size=3*inch_to_mm

#### CCD Detector

In [84]:
Det_xpic=24.0 # microns per pixel
Det_NbPix=2048 # number of pixels per CCD side
Det_size=Det_xpic*Det_NbPix*1e-3 # CCD size in mm, 5 cm or 2 inches

In [85]:
Det_pix_angl=Det_xpic*micr_to_m/Tel_Focal_Length   # in radian

In [86]:
print "pixel viewing angle in arcsec = ", Det_pix_angl*rad_to_deg*arcdeg_to_arcsec

pixel viewing angle in arcsec =  0.392885345233


# Ymax

In [87]:
ymax=1./2.*Filt_D*Tel_Diameter/Tel_Focal_Length*m_to_mm  # mm

In [88]:
print ' ymax = ',ymax,' mm'

 ymax =  2.06035714286  mm


# Resolution 

In [89]:
ResWL=2.*Ronchi_r*ymax

In [90]:
def ComputeWlRes(wl):
    return wl/ResWL

# Atmosphere


In [91]:
file1_in='RT_LS_pp_us_sc_rt_z15.OUT'
file2_in='RT_LS_pp_us_ab_rt_z15_wv40.OUT'
file3_in='RT_LS_pp_us_ab_rt_z15_oz32.OUT'
file4_in='RT_LS_pp_us_sa_rt_z15_wv40.OUT'
file5_in='RT_LS_pp_us_sa_rt_z15_oz32.OUT'

In [92]:
path_atm_rt_us_sc="/Users/dagoret-campagnesylvie/MacOsX/LSST/MyWork/GitHub/PC5AtmosphericExtinction/LibRadTran/simulations/RT/2.0/LS/pp/us/sc/out"

In [93]:
path_atm_rt_us_ab_rt_oz='/Users/dagoret-campagnesylvie/MacOsX/LSST/MyWork/GitHub/PC5AtmosphericExtinction/LibRadTran/simulations/RT/2.0/LS/pp/us/ab/rt/oz/out'
path_atm_rt_us_ab_rt_pwv='/Users/dagoret-campagnesylvie/MacOsX/LSST/MyWork/GitHub/PC5AtmosphericExtinction/LibRadTran/simulations/RT/2.0/LS/pp/us/ab/rt/wv/out'

In [94]:
path_atm_rt_us_sa_rt_oz='/Users/dagoret-campagnesylvie/MacOsX/LSST/MyWork/GitHub/PC5AtmosphericExtinction/LibRadTran/simulations/RT/2.0/LS/pp/us/sa/rt/oz/out'
path_atm_rt_us_sa_rt_pwv='/Users/dagoret-campagnesylvie/MacOsX/LSST/MyWork/GitHub/PC5AtmosphericExtinction/LibRadTran/simulations/RT/2.0/LS/pp/us/sa/rt/wv/out'

In [95]:
sc_rt_path=os.path.join(path_atm_rt_us_sc,file1_in)
abs_rt_pwv_path=os.path.join(path_atm_rt_us_ab_rt_pwv,file2_in)
abs_rt_oz_path=os.path.join(path_atm_rt_us_ab_rt_oz,file3_in)
sa_rt_pwv_path=os.path.join(path_atm_rt_us_sa_rt_pwv,file4_in)
sa_rt_oz_path=os.path.join(path_atm_rt_us_sa_rt_oz,file5_in)

In [96]:
data1=np.loadtxt(sc_rt_path)
data2=np.loadtxt(abs_rt_pwv_path)
data3=np.loadtxt(abs_rt_oz_path)
data4=np.loadtxt(sa_rt_pwv_path)
data5=np.loadtxt(sa_rt_oz_path)

In [97]:
x_rt_1=data1[:,0]   # pure scatering
y_rt_1=data1[:,1]

x_rt_2=data2[:,0]   # pure absorption (H2O = 4mm) 
y_rt_2=data2[:,1]

x_rt_3=data3[:,0]   # pure absorption (OZ = 320 Db) 
y_rt_3=data3[:,1]

x_rt_4=data4[:,0]   # Both absorption and scattering (H2O = 4 mm)
y_rt_4=data4[:,1]

x_rt_5=data5[:,0]   # Both absorption and scattering (OZ = 320 mm)
y_rt_5=data5[:,1]

In [98]:
y_rt_6=y_rt_1*y_rt_2
y_rt_7=y_rt_1*y_rt_3

In [99]:
file1_out=file1_in.split(".")[0]+'.xlsx'
file2_out=file2_in.split(".")[0]+'.xlsx'
file3_out=file3_in.split(".")[0]+'.xlsx'
file4_out=file4_in.split(".")[0]+'.xlsx'
file5_out=file5_in.split(".")[0]+'.xlsx'

In [100]:
df1=pd.DataFrame()
df1["wl"]=x_rt_1
df1["trans_scatt"]=y_rt_1
df1.to_excel(file1_out)

In [101]:
df2=pd.DataFrame()
df2["wl"]=x_rt_2
df2["trans_absH2O4mm"]=y_rt_2
df2.to_excel(file2_out)

In [102]:
df3=pd.DataFrame()
df3["wl"]=x_rt_3
df3["trans_absOz320DU"]=y_rt_3
df3.to_excel(file3_out)

In [103]:
df4=pd.DataFrame()
df4["wl"]=x_rt_4
df4["trans_totH2O4mm"]=y_rt_4
df4.to_excel(file4_out)

In [104]:
df5=pd.DataFrame()
df5["wl"]=x_rt_5
df5["trans_totOz320DU"]=y_rt_5
df5.to_excel(file5_out)